In [ ]:
# answering on context

In [ ]:


# tokenizer = BertTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")
# model = BertForQuestionAnswering.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")


# context = "React is a powerful and widely-used JavaScript library for building user interfaces, especially single-page applications that demand real-time data handling and dynamic user interactions. Developed and maintained by Facebook, React enables developers to create large web applications that can efficiently update and render in response to changing data. A key feature of React is its component-based architecture, which allows developers to break down complex UIs into smaller, reusable pieces, making the codebase more maintainable and scalable. Central to React's functionality is the concept of state, which represents the dynamic parts of an application. State is managed within components and can change over time, driving the UI to re-render and reflect the latest data. This reactive nature, coupled with React's use of a virtual DOM, ensures high performance by minimizing the number of direct updates to the actual DOM, resulting in faster rendering and a smoother user experience. Whether for building simple websites or intricate web applications, React, with its powerful state management and dynamic UI capabilities, has become a top choice for developers aiming to create responsive and interactive user interfaces.",


# question = "What has React become a top choice for developers?"


# inputs = tokenizer(question, context, return_tensors="pt")


# with torch.no_grad():
#     outputs = model(**inputs)


# start_logits = outputs.start_logits
# end_logits = outputs.end_logits


# top_n = 3
# start_indices = torch.topk(start_logits, top_n).indices.squeeze()
# end_indices = torch.topk(end_logits, top_n).indices.squeeze()


# answers = []
# for start_idx in start_indices:
#     for end_idx in end_indices:
#         if end_idx >= start_idx:
#             answer = tokenizer.convert_tokens_to_string(
#                 tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start_idx:end_idx+1])
#             )
#             answers.append(answer)


# print(f"Question: {question}")
# print(f"Possible Answers:")
# for i, answer in enumerate(answers, 1):
#     print(f"{i}. {answer}")

In [8]:
# Generating question

!pip install torch transformers sentence-transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer, util
import numpy as np


model_name = "allenai/t5-small-squad2-question-generation"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


embedder = SentenceTransformer('all-MiniLM-L6-v2')


def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    return output

# Function to filter similar questions
def filter_similar_questions(questions, similarity_threshold=0.75):
    embeddings = embedder.encode(questions, convert_to_tensor=True)
    unique_questions = []

    for i, question in enumerate(questions):
        if i == 0:
            unique_questions.append(question)
        else:
            is_similar = False
            for unique_question in unique_questions:
                unique_embedding = embedder.encode(unique_question, convert_to_tensor=True)
                similarity = util.pytorch_cos_sim(embeddings[i], unique_embedding).item()
                if similarity > similarity_threshold:
                    is_similar = True
                    break
            if not is_similar:
                unique_questions.append(question)

    return unique_questions


questions = run_model(
    """The Transformers library by Hugging Face provides pre-trained models and tools for natural language processing tasks.
    It supports a variety of architectures, including BERT, GPT, T5, and many others, making it a popular choice for developers.""",
    num_beams=15,
    num_return_sequences=15,
    max_length=128,
    early_stopping=True,
    temperature=0.7,
    top_p=0.85,
    top_k=50
)


filtered_questions = filter_similar_questions(questions)
print(filtered_questions)

bert_tokenizer = BertTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")
bert_model = BertForQuestionAnswering.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")


for idx, question in enumerate(filtered_questions, 1):
    print(f"Question {idx}: {question}")

    context = """The Transformers library by Hugging Face provides pre-trained models and tools for natural language processing tasks.
    It supports a variety of architectures, including BERT, GPT, T5, and many others, making it a popular choice for developers."""


    inputs = bert_tokenizer(question, context, return_tensors="pt")

    with torch.no_grad():
        outputs = bert_model(**inputs)


    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    top_n = 3
    start_indices = torch.topk(start_logits, top_n).indices.squeeze()
    end_indices = torch.topk(end_logits, top_n).indices.squeeze()

    answers = []
    for start_idx in start_indices:
        for end_idx in end_indices:
            if end_idx >= start_idx:
                answer = bert_tokenizer.convert_tokens_to_string(
                    bert_tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start_idx:end_idx+1])
                )
                answers.append(answer)


    print(f"Question: {question}")
    print(f"Possible Answers:")
    for i, answer in enumerate(answers, 1):
        print(f"{i}. {answer}")
        print()


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


['What library supports a variety of architectures?', 'What is the name of the Transformers library?', "What does Hugging Face's Transformers library support?", 'How many architectures does Hugging Face support?', 'What library provides pre-trained models and tools?']


Some weights of the model checkpoint at deepset/bert-large-uncased-whole-word-masking-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question 1: What library supports a variety of architectures?
Question: What library supports a variety of architectures?
Possible Answers:
1. the transformers library

2. the transformers

3. the transformers library by hugging face

4. transformers library

5. transformers

6. transformers library by hugging face

7. [CLS] what library supports a variety of architectures ? [SEP] the transformers library

8. [CLS] what library supports a variety of architectures ? [SEP] the transformers

9. [CLS] what library supports a variety of architectures ? [SEP] the transformers library by hugging face

Question 2: What is the name of the Transformers library?
Question: What is the name of the Transformers library?
Possible Answers:
1. hugging face

2. [CLS] what is the name of the transformers library ? [SEP] the transformers library by hugging face

3. [CLS]

4. [CLS] what is the name of the transformers library ? [SEP] the transformers library

5. transformers library by hugging face

6. tra

In [ ]:
tokenizer = BertTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")
model = BertForQuestionAnswering.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")


context = "React is a powerful and widely-used JavaScript library for building user interfaces, especially single-page applications that demand real-time data handling and dynamic user interactions. Developed and maintained by Facebook, React enables developers to create large web applications that can efficiently update and render in response to changing data. A key feature of React is its component-based architecture, which allows developers to break down complex UIs into smaller, reusable pieces, making the codebase more maintainable and scalable. Central to React's functionality is the concept of state, which represents the dynamic parts of an application. State is managed within components and can change over time, driving the UI to re-render and reflect the latest data. This reactive nature, coupled with React's use of a virtual DOM, ensures high performance by minimizing the number of direct updates to the actual DOM, resulting in faster rendering and a smoother user experience. Whether for building simple websites or intricate web applications, React, with its powerful state management and dynamic UI capabilities, has become a top choice for developers aiming to create responsive and interactive user interfaces.",


question = "What has React become a top choice for developers?"


inputs = tokenizer(question, context, return_tensors="pt")


with torch.no_grad():
    outputs = model(**inputs)


start_logits = outputs.start_logits
end_logits = outputs.end_logits


top_n = 3
start_indices = torch.topk(start_logits, top_n).indices.squeeze()
end_indices = torch.topk(end_logits, top_n).indices.squeeze()


answers = []
for start_idx in start_indices:
    for end_idx in end_indices:
        if end_idx >= start_idx:
            answer = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start_idx:end_idx+1])
            )
            answers.append(answer)


print(f"Question: {question}")
print(f"Possible Answers:")
for i, answer in enumerate(answers, 1):
    print(f"{i}. {answer}")

In [ ]:
# Simillar to q and a model more accurate

In [13]:

# from transformers import BertTokenizer, BertForQuestionAnswering
# import torch
# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer


# tokenizer = BertTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")
# model = BertForQuestionAnswering.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")


# embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')


# contexts = {
#     1: """
#     The Transformers library by Hugging Face provides pre-trained models and tools for natural language processing tasks.
#     It supports a variety of architectures, including BERT, GPT, T5, and many others, making it a popular choice for developers.
#     """,
#     2: """
#     Python is a versatile programming language that is widely used for web development, data analysis, artificial intelligence,
#     scientific computing, and automation. Its simple syntax and readability make it a popular choice among beginners and professionals alike.
#     """,
#     3: """
#     The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It was named after the engineer Gustave Eiffel,
#     whose company designed and built the tower. It is one of the most recognizable structures in the world.
#     """,
#     4: """
#     Naruto Uzumaki is the main character in the anime and manga series "Naruto." He is a young ninja who wants to become the strongest leader of his village, the Hokage. Although he had a difficult childhood and was often treated poorly because of the demon fox inside him, Naruto never gives up on his dream. His story is about making friends, facing challenges, and growing as a person. Through his journey, he shows that hard work and never giving up are important.
#     """,
#     5: """
#     The International Space Station (ISS) is a space station, or a habitable artificial satellite, in low Earth orbit. The ISS is a joint project of NASA, Roscosmos, JAXA, ESA, and CSA. It serves as a microgravity and space environment research laboratory in which scientific research is conducted in astrobiology, astronomy, meteorology, physics, and other fields.
#     """,
#     6: """
#     Machine learning is a subset of artificial intelligence that focuses on the development of algorithms that allow computers to learn from and make predictions or decisions based on data. It is widely used in various industries, from finance to healthcare, for tasks such as image recognition, fraud detection, and personalized recommendations.
#     """,
#     7: """
#     Marie Curie was a Polish and naturalized-French physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize and remains the only person to win Nobel Prizes in two different scientific fields: Physics and Chemistry. Her discoveries laid the foundation for the development of X-ray machines and cancer treatments.
#     """,
#     8: """
#     The Amazon rainforest is the largest tropical rainforest in the world, covering much of northwestern Brazil and extending into Colombia, Peru, and other South American countries. It is known for its biodiversity, containing millions of species of plants, insects, birds, and other wildlife. The Amazon plays a crucial role in regulating the global climate and is often referred to as the "lungs of the Earth."
#     """,
#     9: """
#     Albert Einstein was a theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (the other being quantum mechanics). His work is also known for its influence on the philosophy of science. Einstein's equation E=mc^2, which states that energy and mass are interchangeable, is one of the most famous equations in physics.
#     """,
#     10: """
#     The Mona Lisa is a half-length portrait painting by the Italian artist Leonardo da Vinci. Considered an archetypal masterpiece of the Italian Renaissance, it has been described as "the most visited, most talked about, and most sung about piece of art in the world." The painting's enigmatic expression, often described as enigmatic, has captivated viewers for centuries.
#     """,
#     11: """
#     Quantum computing is an area of computing focused on developing computer technology based on the principles of quantum theory. Quantum computers use quantum bits or qubits, which can represent and store information in more complex ways than classical bits. Quantum computing has the potential to solve complex problems much faster than classical computers, particularly in fields such as cryptography and materials science.
#     """,
#     12: """
#     The Great Wall of China is a series of fortifications that were built across the historical northern borders of China to protect and consolidate territories of Chinese states and empires against various nomadic groups. It is the longest wall in the world and is considered one of the greatest architectural achievements in history. The wall stretches over 13,000 miles and was built over several dynasties.
#     """,
#     13: """
#     Nikola Tesla was a Serbian-American inventor, electrical engineer, mechanical engineer, and futurist who is best known for his contributions to the design of the modern alternating current (AC) electricity supply system. Tesla's innovations in electrical engineering laid the foundation for much of the technology we use today, including radio, television, and modern power transmission systems.
#     """,
#     14: """
#     The Pythagorean Theorem is a fundamental principle in geometry that states that in a right-angled triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. This theorem is used in various fields, including physics, engineering, computer science, and architecture.
#     """,
#     15: """
#     The Internet is a global network of interconnected computer networks that use the Internet protocol suite (TCP/IP) to communicate with each other. It has revolutionized communication, commerce, education, and entertainment. The Internet allows people to connect and share information across the world in real time, making it one of the most significant technological advancements in human history.
#     """
# }



# context_embeddings = embedder.encode(list(contexts.values()))


# print("Available contexts:")
# for key, value in contexts.items():
#     print(f"{key}: {value.strip()[:60]}...")


# question = input("Enter your question: ")


# question_embedding = embedder.encode([question])[0]


# similarity_scores = cosine_similarity([question_embedding], context_embeddings)[0]


# most_relevant_index = similarity_scores.argmax()
# context = list(contexts.values())[most_relevant_index]


# inputs = tokenizer(question, context, return_tensors="pt")


# with torch.no_grad():
#     outputs = model(**inputs)


# answer_start = torch.argmax(outputs.start_logits)
# answer_end = torch.argmax(outputs.end_logits) + 1


# answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))


# print(f"\nSelected Context: {list(contexts.keys())[most_relevant_index]}")
# print(f"Context: {context.strip()[:200]}...")
# print(f"\nQuestion: {question}")
# print(f"Answer: {answer}")


Some weights of the model checkpoint at deepset/bert-large-uncased-whole-word-masking-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Available contexts:
1: The Transformers library by Hugging Face provides pre-traine...
2: Python is a versatile programming language that is widely us...
3: The Eiffel Tower is a wrought-iron lattice tower on the Cham...
4: Naruto Uzumaki is the main character in the anime and manga ...
5: The International Space Station (ISS) is a space station, or...
6: Machine learning is a subset of artificial intelligence that...
7: Marie Curie was a Polish and naturalized-French physicist an...
8: The Amazon rainforest is the largest tropical rainforest in ...
9: Albert Einstein was a theoretical physicist who developed th...
10: The Mona Lisa is a half-length portrait painting by the Ital...
11: Quantum computing is an area of computing focused on develop...
12: The Great Wall of China is a series of fortifications that w...
13: Nikola Tesla was a Serbian-American inventor, electrical eng...
14: The Pythagorean Theorem is a fundamental principle in geomet...
15: The Internet is a global network 

In [ ]:
followup questions

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "EleutherAI/gpt-neo-1.3B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_follow_up_question_gpt_neo(previous_answer):

    prompt = (
        "Given the user's previous answer, generate a relevant follow-up question. "
        f"Previous answer: {previous_answer}\nFollow-up question:"
    )


    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    output = model.generate(
        input_ids,
        max_length=100,
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        temperature=0.7
    )


    follow_up_question = tokenizer.decode(output[0], skip_special_tokens=True)

    follow_up_question = follow_up_question[len(prompt):].strip()
    return follow_up_question


previous_answer = "Iam currently learning devops for deploying the full stack projects and ml"
follow_up_question = generate_follow_up_question_gpt_neo(previous_answer)
print(f"Follow-up Question: {follow_up_question}")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Question 1: questions: The Moon is Earth's only natural satellite and the fifth largest moon in the Solar System.
Question 2: Fragen: The Moon is Earth's only natural satellite and the fifth largest moon in the Solar System. It is the closest celestial body to Earth.
Question 3: Fragen: The Moon is Earth's only natural satellite and the fifth largest moon in the Solar System.
